## Test prod API notebook

This notebook can be used to test Vauban API. You can find a set of examples using OpenAI client or directly requests.

Available models:

- `fast-v1`: meta-llama/Llama-3.1-8B
- `chat-v1`: nvidia/Llama-3.1-Nemotron-70B-Instruct
- `embeddings-v1`: Alibaba-NLP/gte-Qwen2-7B-instruct

`chat-v1` is only available in TEST and PROD api keys. If you use `chat-v1` for DEV API key it will automatically use `fast-v1` model.

Currently, for chat models with TEST and PROD api keys you can only use async mode.

In [ ]:
# Put your API key here

# DEV
API_KEY = ""

# TEST
# API_KEY = ""

# PROD
# API_KEY = ""

In [4]:
# Put the base URL (prod / dev or local)

BASE_URL = "https://api.vauban.cloud/v1"

#### Embedding endpoint

With OpenAI client

In [ ]:
# With OpenAI

from openai import OpenAI

client = OpenAI(
    api_key=API_KEY,
    base_url=BASE_URL
)

response = client.embeddings.create(
    input="This is an amazing input",
    model="embeddings-v1"
)

print(response)

With OpenAI client async mode

In [7]:
# With OpenAI

from openai import OpenAI

client = OpenAI(
    api_key=API_KEY,
    base_url=BASE_URL
)

response = client.embeddings.create(
    input="This is an amazing input",
    model="embeddings-v1",
    extra_body={"deferred": True}
)

print(response)
job_id = response.id

CreateEmbeddingResponse(data=[Embedding(embedding=[], index=0, object='embedding')], model='embeddings-v1', object='list', usage=Usage(prompt_tokens=0, total_tokens=0, completion_tokens=0), id='13815214-3341-4fe3-8e8c-09f2c2c809bc')


In [ ]:
# Check job status

import requests

url = f"{BASE_URL}/jobs/out/{job_id}"
headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {API_KEY}"
}

response = requests.get(url, headers=headers)
response.json()

In [ ]:
# Check job output

import requests

url = f"{BASE_URL}/jobs/out/{job_id}?status_only=false"
headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {API_KEY}"
}

response = requests.get(url, headers=headers)
response.json()

With requests

In [ ]:
# With requests
import requests

url = f"{BASE_URL}/embeddings"
headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {API_KEY}"
}
data = {
    "input": "Hello how are you ?",
}

response = requests.post(url, headers=headers, json=data)

print(response.json())

With requests async mode

In [11]:
# With requests
import requests

url = f"{BASE_URL}/embeddings"
headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {API_KEY}"
}
data = {
    "input": "Hello how are you ?",
    "deferred": True
}

response = requests.post(url, headers=headers, json=data)
resp_dict = response.json()

job_id = resp_dict.get('id')
resp_dict

{'id': '9c73a8e6-2d3d-47bf-ba6f-e22e5e42cbca',
 'object': 'list',
 'data': [{'object': 'embedding', 'embedding': [], 'index': 0}],
 'model': 'embeddings-v1',
 'usage': {'prompt_tokens': 0, 'completion_tokens': 0, 'total_tokens': 0}}

In [12]:
# Check job status

import requests

url = f"{BASE_URL}/jobs/job/{job_id}"
headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {API_KEY}"
}

response = requests.get(url, headers=headers)
response.json()

{'status': 'Queued'}

In [13]:
# Check job input

import requests

url = f"{BASE_URL}/jobs/job/{job_id}?status_only=false"
headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {API_KEY}"
}

response = requests.get(url, headers=headers)
response.json()

{'model': 'embeddings-v1',
 'input': ['Hello how are you ?'],
 'user': None,
 'deferred': True,
 'job_name': None,
 'id': '9c73a8e6-2d3d-47bf-ba6f-e22e5e42cbca',
 'job_type': 'embeddings',
 'status': 'Queued'}

In [ ]:
# Check job output

import requests

url = f"{BASE_URL}/jobs/out/{job_id}?status_only=false"
headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {API_KEY}"
}

response = requests.get(url, headers=headers)
response.json()

#### Rerank endpoint

Sync

In [24]:
import requests

url = f"{BASE_URL}/rerank"

headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {API_KEY}"
}

data = {
    "query": "What is the capital of the United States?",
    "model": "embeddings-v1",
    "top_n": 4,
    "documents": [
        "Carson City is the capital city of the American state of Nevada.",
        "The Commonwealth of the Northern Mariana Islands is a group of islands in the Pacific Ocean. Its capital is Saipan.",
        "Washington, D.C. (also known as simply Washington or D.C., and officially as the District of Columbia) is the capital of the United States. It is a federal district.",
        "Capitalization or capitalisation in English grammar is the use of a capital letter at the start of a word. English usage varies from capitalization in other languages.",
        "Capital punishment has existed in the United States since beforethe United States was a country. As of 2017, capital punishment is legal in 30 of the 50 states."
    ]
}

response = requests.post(url, headers=headers, json=data)
print(response.json())

{'id': '977b5925-b70e-43a1-b674-7aeda9c0d529', 'results': [{'index': 2, 'relevance_score': 0.590172147354083}, {'index': 0, 'relevance_score': 0.4787417495551361}, {'index': 4, 'relevance_score': 0.3494323799858749}, {'index': 1, 'relevance_score': 0.2909402671344156}], 'meta': {'api_version': {'version': '1', 'is_experimental': False}, 'billed_units': {'search_units': 1}}, 'usage': {'prompt_tokens': 0, 'completion_tokens': 0, 'total_tokens': 0}}


Async mode

In [ ]:
import requests

url = f"{BASE_URL}/rerank"

headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {API_KEY}"
}

data = {
    "query": "I want details about a French dessert",
    "model": "embeddings-v1",
    "top_n": 4,
    "documents": [
        "How to make a Panna Cotta",
        "Home made Bubble tea",
        "Let's make together some Crêpes",
        "Home made peanut butter"
    ],
    # Just add this line for the async mode
    "deferred": True
}

response = requests.post(url, headers=headers, json=data)
print(response)
resp_dict = response.json()

job_id = resp_dict.get('id')
resp_dict

In [ ]:
# Check job input

import requests

url = f"{BASE_URL}/jobs/job/{job_id}?status_only=false"
headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {API_KEY}"
}

response = requests.get(url, headers=headers)
response.json()

In [ ]:
# Check job output

import requests

url = f"{BASE_URL}/jobs/out/{job_id}?status_only=false"
headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {API_KEY}"
}

response = requests.get(url, headers=headers)
response.json()

#### Chat endpoint

With OpenAI client

In [ ]:
# With OpenAI

from openai import OpenAI

client = OpenAI(
    api_key=API_KEY,
    base_url=BASE_URL
)

prompt = """
Tell me why France has the best food
"""

response = client.chat.completions.create(
    model="chat-v1",
    max_tokens=800,
    messages=[
        {
            "role": "user",
            "content": prompt
        }
    ]
)

print(response)

In [17]:
print(response.choices[0].message.content)

France is indeed renowned for its exquisite cuisine, but whether it has the "best" food is subjective and open to personal opinion. However, there are many reasons why French cuisine is so highly regarded:

1. **Rich culinary history**: French cuisine has a long and storied history, with roots dating back to the Middle Ages. It has been influenced by various cultures, including the Romans, the Normans, and the Renaissance-era aristocrats.
2. **Emphasis on technique**: French cuisine places a strong emphasis on proper technique, using high-quality ingredients, and attention to presentation. This focus on craftsmanship has earned French chefs a reputation for excellence.
3. **Regional diversity**: France is home to numerous distinct culinary regions, each with its unique flavors, ingredients, and specialties. From the buttery pastries of Brittany to the spicy cuisine of the southwest, there's a wealth of flavors to explore.
4. **Love for local ingredients**: French cuisine celebrates the

With OpenAI client async mode

In [ ]:
# With OpenAI

from openai import OpenAI

client = OpenAI(
    api_key=API_KEY,
    base_url=BASE_URL
)

prompt = """
Tell me why France has the best food
"""

response = client.chat.completions.create(
    model="fast-v1",
    max_tokens=800,
    messages=[
        {
            "role": "user",
            "content": prompt
        }
    ],
    extra_body={"deferred": True}
)

print(response)
job_id = response.id

ChatCompletion(id='bb1b9b01-a0a4-4cc3-a750-694cf0648793', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None))], created=1735207811, model='chat-v1', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=0, prompt_tokens=0, total_tokens=0, completion_tokens_details=None, prompt_tokens_details=None), sources=[])


In [ ]:
# Check job status

import requests

url = f"{BASE_URL}/jobs/out/{job_id}"
headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {API_KEY}"
}

response = requests.get(url, headers=headers)
response.json()

{'status': 'Done'}

In [21]:
# Check job output

import requests

url = f"{BASE_URL}/jobs/out/{job_id}?status_only=false"
headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {API_KEY}"
}

response = requests.get(url, headers=headers)
response.json()

{'id': 'bb1b9b01-a0a4-4cc3-a750-694cf0648793',
 'object': 'chat.completion',
 'created': 1735207819,
 'model': 'chat-v1',
 'choices': [{'index': 0,
   'message': {'role': 'assistant',
    'content': 'France is indeed renowned for its rich culinary culture and the quality of its cuisine. There are several reasons why French food is so highly regarded:\n\n1. **History and Tradition**: French cuisine has a long and storied history, dating back to the Middle Ages. The country\'s cooking techniques and recipes have evolved over time, influenced by various cultures and culinary traditions.\n2. **High-Quality Ingredients**: France is famous for its fresh produce, dairy products, meats, and seafood. The country\'s emphasis on using high-quality, seasonal ingredients helps to create dishes that are both flavorful and visually appealing.\n3. **Culinary Expertise**: French chefs are known for their technical skill and attention to detail, which allows them to prepare dishes that are both visually

With requests

In [7]:
# With requests

import requests

url = f"{BASE_URL}/chat/completions"
headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {API_KEY}"
}
data = {
    "model": "chat-v1",
    "max_tokens": 2000,
    "messages": [
        {
            "role": "system",
            "content": "You have to answer user requests and follow carefully prompt requirements."
        },
        {
            "role": "user",
            "content": "Is it possible to land on the sun? (answer in 2 sentences with a short explanation)"
        }
    ]
}

response = requests.post(url, headers=headers, json=data)
print(response.json()['choices'][0]['message']['content'])

It is not possible for any spacecraft to land on the sun due to its incredibly high surface temperature of about 5500°C (10,000°F), which would vaporize any material that came into contact with it. Additionally, the sun's intense gravity would crush any spacecraft that got too close, making it impossible for it to survive the journey.


In [3]:
import requests

url = f"{BASE_URL}/chat/completions"
headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {API_KEY}"
}
data = {
    "model": "chat-v1",
    "max_tokens": 2000,
    "messages": [
        {
            "role": "system",
            "content": "You have to answer user requests and follow carefully prompt requirements."
        },
        {
            "role": "user",
            "content": "Is it possible to land on the sun? (answer in 2 sentences with a short explanation)"
        }
    ],
    "deferred": True
}

response = requests.post(url, headers=headers, json=data)
response_dict = response.json()

job_id = response_dict['id']
response_dict

{'id': '081004ec-48a6-4296-a5ab-c211bce7ef65',
 'object': 'chat.completion',
 'created': 1734425016,
 'model': 'chat-v1',
 'choices': [{'index': 0,
   'message': {'role': 'assistant', 'content': ''},
   'finish_reason': 'stop'}],
 'usage': {'prompt_tokens': 0, 'completion_tokens': 0, 'total_tokens': 0},
 'sources': []}

In [ ]:
# Check job status

import requests

url = f"{BASE_URL}/jobs/job/{job_id}?status_only=false"
headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {API_KEY}"
}

response = requests.get(url, headers=headers)
response.json()

{'messages': ['Is it possible to land on the sun? (answer in 2 sentences with a short explanation)'],
 'max_tokens': 2000,
 'stream': False,
 'stop': None,
 'n': 1,
 'temperature': 1.0,
 'top_k': 40,
 'top_p': 0.9,
 'presence_penalty': 0.0,
 'frequency_penalty': 0.0,
 'repetition_penalty': 1.0,
 'seed': 0,
 'model': 'chat-v1',
 'deferred': True,
 'rag': False,
 'system': 'You have to answer user requests and follow carefully prompt requirements.',
 'job_type': 'chat',
 'status': 'Done'}

In [6]:
# Check job output

import requests

url = f"{BASE_URL}/jobs/out/{job_id}?status_only=false"
headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {API_KEY}"
}

response = requests.get(url, headers=headers)
response.json()

{'id': '081004ec-48a6-4296-a5ab-c211bce7ef65',
 'object': 'chat.completion',
 'created': 1734425030,
 'model': 'chat-v1',
 'choices': [{'index': 0,
   'message': {'role': 'assistant',
    'content': "It is not possible for any object, including a spacecraft, to land on the sun because the sun's surface temperature is about 5,500°C (10,000°F), which is far beyond the melting point of any material, and the intense gravitational forces would cause any object to be torn apart. Additionally, the sun's gravity is so strong that even if a spacecraft could somehow withstand the heat, it would be pulled apart by the sun's tidal forces before it could even reach the surface."},
   'finish_reason': 'stop'}],
 'usage': {'prompt_tokens': 20, 'completion_tokens': 100, 'total_tokens': 120},
 'sources': [],
 'status': 'Done'}

#### RAG

#### Push documents

You can directly push your documents (.pdf, .md, .docx)

In [15]:
import requests

# For multiple files upload
folder = "client1"
url = f"{BASE_URL}/rag/documents?folder={folder}"
files = [
    ("files", ("Diagnosis_and_Management_of_Heart_Failure.pdf", open("../data/Diagnosis_and_Management_of_Heart_Failure.pdf", "rb")))
]
headers = {
    "Authorization": f"Bearer {API_KEY}"
}

response = requests.post(url, files=files, headers=headers)
print(f"Status code: {response.status_code}")
print(f"Response: {response.json()}")

Status code: 200
Response: {'message': 'Documents pushed'}


You can also directly push your markdown

In [17]:
import requests

url = f"{BASE_URL}/rag/markdown"
headers = {
    "Authorization": f"Bearer {API_KEY}"
}
body = {
    "folder": "client1",
    "filename": "test.md",
    "filecontent": "# Title1\n## Title2\nTest document1\n## Title3\nTest document2\n### Title4\nHello"
}

response = requests.post(url, headers=headers, json=body)
print(f"Status code: {response.status_code}")
print(f"Response: {response.json()}")

Status code: 200
Response: {'message': 'Documents pushed'}


#### List documents

In [18]:
import requests

url = f"https://api.vauban.cloud/v1/rag/documents"
headers = {
    "Authorization": f"Bearer {API_KEY}"
}

response = requests.get(url, headers=headers)
print(response.json())

{'env_type': 'DEV', 'files': [{'name': 'nuclear-astrophysics.pdf', 'file_type': 'pdf'}, {'name': 'test.md', 'file_type': 'md'}, {'name': 'Diagnosis_and_Management_of_Heart_Failure.pdf', 'file_type': 'pdf'}]}


#### Delete documents

In [19]:
import requests

url = f"{BASE_URL}/rag/documents"
headers = {
    "Authorization": f"Bearer {API_KEY}"
}
body = {
    # Filenames you can get in the list documents endpoint
    'filenames': ['test.md']
}

response = requests.delete(url, headers=headers, json=body)
print(response.json())

{'message': 'Documents deleted successfully'}


##### Query

With OpenAI client sync version

In [ ]:
# With OpenAI

from openai import OpenAI

client = OpenAI(
    api_key=API_KEY,
    base_url=BASE_URL
)

query = """
What associations did the study identify between non-alcoholic fatty liver disease (NAFLD),
liver fibrosis, and cardiac structural or functional disturbances in an Asian population?
Highlight the key findings regarding epicardial fat volume (EFV), diastolic function, and left atrial mechanics.
"""

response = client.chat.completions.create(
    model="fast-v1",
    max_tokens=1000,
    messages=[
        {
            "role": "user",
            "content": query
        }
    ],
    extra_body={"rag": True}
)

print(response.choices[0].message.content)
print(response.usage)
print(response.sources)

The study identified several associations between non-alcoholic fatty liver disease (NAFLD), liver fibrosis, and cardiac structural or functional disturbances in an Asian population.

**Associations with NAFLD and Liver Fibrosis:**

* The study found that individuals with NAFLD and liver fibrosis had higher levels of ectopic fat depots, including epicardial fat volume (EFV), compared to those without NAFLD or liver fibrosis.
* NAFLD and liver fibrosis were associated with cardiac structural and functional disturbances, including diastolic dysfunction.

**Key Findings:**

* **Epicardial Fat Volume (EFV):** The study found that EFV was significantly higher in individuals with NAFLD and liver fibrosis, particularly in those with a high fibrosis score. This suggests that excessive visceral fat can accumulate as ectopic fat depots over vital organs, including the heart.
* **Diastolic Function:** The study found that diastolic function was impaired in individuals with NAFLD and liver fibrosi

With requests Sync version

In [21]:
query = """
What associations did the study identify between non-alcoholic fatty liver disease (NAFLD),
liver fibrosis, and cardiac structural or functional disturbances in an Asian population?
Highlight the key findings regarding epicardial fat volume (EFV), diastolic function, and left atrial mechanics.
"""

url = f"{BASE_URL}/chat/completions"
headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {API_KEY}"
}
body = {
    "model": "chat-v1",
    "max_tokens": 1000,
    "messages": [
        {
            "role": "user",
            "content": query
        }
    ],
    "rag": True
}
response = requests.post(url, json=body, headers=headers)
print(response.json()['choices'][0]['message']['content'])

Based on the provided document, here are the key associations and findings identified in the study:

**Associations:**

* The study found associations between non-alcoholic fatty liver disease (NAFLD) and liver fibrosis with:
	+ Epicardial Adipose Tissue Volume (EATV)
	+ Atrial Deformation Mechanics (i.e., diastolic function and left atrial mechanics)
* These associations were observed in a large Asian population free from clinical heart failure.

**Key Findings:**

* **Epicardial Fat Volume (EFV):** The study found that:
	+ EFV was significantly higher in participants with NAFLD and liver fibrosis compared to those without NAFLD and liver fibrosis.
	+ The association between EFV and NAFLD and liver fibrosis was stronger in participants with higher fibrosis scores.
* **Diastolic Function:** The study found that:
	+ Participants with NAFLD and liver fibrosis had lower diastolic function (atrial deformation mechanics) compared to those without NAFLD and liver fibrosis.
* **Left Atrial Me

With OpenAI client async version

In [24]:
# With OpenAI

from openai import OpenAI

client = OpenAI(
    api_key=API_KEY,
    base_url=BASE_URL
)

query = """
What associations did the study identify between non-alcoholic fatty liver disease (NAFLD),
liver fibrosis, and cardiac structural or functional disturbances in an Asian population?
Highlight the key findings regarding epicardial fat volume (EFV), diastolic function, and left atrial mechanics.
"""

response = client.chat.completions.create(
    model="fast-v1",
    max_tokens=1000,
    messages=[
        {
            "role": "user",
            "content": query
        }
    ],
    extra_body={"rag": True, "deferred": True}
)

print(response)

ChatCompletion(id='27a0e565-653b-44c6-a102-14671a58b15e', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None))], created=1735210060, model='fast-v1', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=0, prompt_tokens=0, total_tokens=0, completion_tokens_details=None, prompt_tokens_details=None), sources=[])


With requests async version

In [25]:
query = """
What associations did the study identify between non-alcoholic fatty liver disease (NAFLD),
liver fibrosis, and cardiac structural or functional disturbances in an Asian population?
Highlight the key findings regarding epicardial fat volume (EFV), diastolic function, and left atrial mechanics.
"""

url = f"{BASE_URL}/chat/completions"
headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {API_KEY}"
}
body = {
    "model": "chat-v1",
    "max_tokens": 1000,
    "messages": [
        {
            "role": "user",
            "content": query
        }
    ],
    "rag": True,
    "deferred": True # For async mode
}
response = requests.post(url, json=body, headers=headers)
job_id = response.json()['id']
response.json()

{'id': 'a8357688-7f41-4408-bb3a-34fd42efd217',
 'object': 'chat.completion',
 'created': 1735210079,
 'model': 'chat-v1',
 'choices': [{'index': 0,
   'message': {'role': 'assistant', 'content': ''},
   'finish_reason': 'stop'}],
 'usage': {'prompt_tokens': 0, 'completion_tokens': 0, 'total_tokens': 0},
 'sources': []}

In [26]:
# Check job status

import requests

url = f"{BASE_URL}/jobs/job/{job_id}?status_only=false"
headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {API_KEY}"
}

response = requests.get(url, headers=headers)
response.json()

{'messages': ['\nWhat associations did the study identify between non-alcoholic fatty liver disease (NAFLD),\nliver fibrosis, and cardiac structural or functional disturbances in an Asian population?\nHighlight the key findings regarding epicardial fat volume (EFV), diastolic function, and left atrial mechanics.\n'],
 'max_tokens': 1000,
 'stream': False,
 'stop': None,
 'n': 1,
 'temperature': 1.0,
 'top_k': 40,
 'top_p': 0.9,
 'presence_penalty': 0.0,
 'frequency_penalty': 0.0,
 'repetition_penalty': 1.0,
 'seed': 0,
 'model': 'chat-v1',
 'deferred': True,
 'rag': True,
 'job_type': 'chat',
 'status': 'Queued'}

In [29]:
# Check job output

import requests

url = f"{BASE_URL}/jobs/out/{job_id}?status_only=false"
headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {API_KEY}"
}

response = requests.get(url, headers=headers)
response.json()

{'id': 'a8357688-7f41-4408-bb3a-34fd42efd217',
 'object': 'chat.completion',
 'created': 1735210096,
 'model': 'chat-v1',
 'choices': [{'index': 0,
   'message': {'role': 'assistant',
    'content': 'Based on the provided document, the study identified the following associations between non-alcoholic fatty liver disease (NAFLD), liver fibrosis, and cardiac structural or functional disturbances in an Asian population:\n\n1. **Epicardial Fat Volume (EFV):** The study found a positive association between NAFLD and liver fibrosis with increased epicardial fat volume (EFV). Specifically, the EFV was significantly higher in participants with fatty liver and high fibrosis score compared to those with non-fatty liver.\n2. **Diastolic Function:** The study reported that NAFLD and liver fibrosis were associated with impaired diastolic function. Participants with fatty liver and high fibrosis score had reduced diastolic function compared to those with non-fatty liver.\n3. **Left Atrial Mechanics: